In [6]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

import pandas as pd
import scipy
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

import os
import pandas as pd
import soundfile as sf
import numpy as np
import math
import scipy.io.wavfile, scipy.signal
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler


In [1]:
from sys import platform

In [3]:
if platform == "linux" or platform == "linux2":
    # linux
    path='/home/vedant/projects/'
elif platform == "darwin":
    # OS X
    path='/Users/vedant/Desktop/Programming/'

In [5]:
cols=['video_id', 'start_time', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']
       
d=np.load(path+'ScreamDetection/resources/working_data/vocal_only_features.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(path+'/ScreamDetection/resources/working_data/dataset/lookup_new.csv')

df.drop('vggish',axis=1,inplace=True)
df


NameError: name 'np' is not defined

In [3]:
feature_df=df
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(4,'label_mapped',mapping)

In [4]:
cols=['video_id', 'start_time', 'mid_ts', 'label', 'label_mapped',
       'average_zcr', 'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std', 'centroid_mean', 'centroid_std',
       'contrast_mean', 'contrast_std', 'flatness_mean', 'flatness_std',
       'rolloff_mean', 'rolloff_std', 'rms_mean', 'rms_std']
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='not minority',random_state=0)
from collections import Counter
X = feature_df.to_numpy()
y=feature_df[['label_mapped']].to_numpy()

X_under, y_under = undersample.fit_resample(X, y)

undersampled_data = pd.DataFrame(X_under,columns=cols)
undersampled_data['label_mapped'] = y_under
#print(undersampled_data)


from sklearn.model_selection import GroupShuffleSplit
train_inds, test_inds = next(GroupShuffleSplit(test_size=.2, n_splits=2, random_state = 42).split(lut, groups=lut['band_name']))

train = lut.iloc[train_inds]
test = lut.iloc[test_inds]

train_ids = train['video_id'].to_numpy()
test_ids = test['video_id'].to_numpy()

#df_final = df
df_final = undersampled_data
train = df_final[df_final.video_id.isin(train_ids)]
test = df_final[df_final.video_id.isin(test_ids)]

In [21]:
test

,video_id,start_time,mid_ts,label,label_mapped,average_zcr,zcr_stddev,mfcc1_mean,mfcc2_mean,mfcc3_mean,...,centroid_mean,centroid_std,contrast_mean,contrast_std,flatness_mean,flatness_std,rolloff_mean,rolloff_std,rms_mean,rms_std
1139,C_voh9WFbsM,42.0,42.5,clean,0,0.172997,0.166189,-444.375642,96.314205,27.460796,...,3992.732704,3135.604591,23.991234,9.153312,0.000585,0.001659,5831.785426,4239.622671,0.063884,0.057711
1140,C_voh9WFbsM,42.5,43.0,clean,0,0.042323,0.028615,-350.112597,149.637213,21.476153,...,1708.31053,997.238817,25.900979,9.929306,0.00002,0.000049,3257.453866,2678.178353,0.099895,0.039792
1141,C_voh9WFbsM,43.0,43.5,clean,0,0.059885,0.029756,-323.834241,136.218839,-21.22201,...,2346.798948,1000.065291,27.530038,9.075277,0.000043,0.000124,4464.055765,2663.812881,0.088009,0.038784
1142,C_voh9WFbsM,43.5,44.0,clean,0,0.058858,0.022372,-288.370695,154.851727,-45.016354,...,2198.118207,740.634754,29.096409,8.411008,0.000005,0.000006,3964.831964,1733.225847,0.103342,0.032928
1143,C_voh9WFbsM,44.0,44.5,clean,0,0.042402,0.018413,-342.113167,160.452053,10.583046,...,1764.219737,950.079016,28.958112,8.028476,0.000013,0.000054,3512.634698,2878.391788,0.102589,0.037338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7369,k27N-jRofrM,65.0,65.5,no_vocals,2,0.235436,0.136577,-534.818986,64.164528,20.941405,...,5787.883061,3302.060978,18.468572,7.713927,0.008837,0.007226,7994.016029,4552.837613,0.035372,0.057372
7370,-2WqQY_xSSM,8.5,9.0,no_vocals,2,0.274246,0.055084,-978.894729,-8.647591,-5.031375,...,8607.284719,616.500599,16.212704,7.810611,0.021439,0.014064,16455.079809,1928.137745,0.000036,0.000024
7375,ainbICPRV8Y,8.5,9.0,no_vocals,2,0.311961,0.02979,-917.588282,-7.008107,-6.108289,...,8846.89522,704.270328,16.615456,9.232447,0.013054,0.007405,16563.240841,4407.436675,0.000964,0.002448
7380,hwxTEcHnC1o,113.0,113.5,no_vocals,2,0.049894,0.004077,-794.072458,143.375389,36.938537,...,2059.762136,489.139064,19.793148,5.781668,0.000738,0.000856,2458.497777,1587.600418,0.000661,0.000465


In [6]:
train.columns

Index(['video_id', 'start_time', 'mid_ts', 'label', 'label_mapped',
       'average_zcr', 'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc

In [7]:
features=['average_zcr', 'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std', 'centroid_mean', 'centroid_std',
       'contrast_mean', 'contrast_std', 'flatness_mean', 'flatness_std',
       'rolloff_mean', 'rolloff_std', 'rms_mean', 'rms_std']
x_train = train[features].to_numpy()
y_train_hot = to_categorical(train['label_mapped'].to_numpy())

x_test = test[features].to_numpy()
y_test_hot = to_categorical(test['label_mapped'].to_numpy())

In [8]:
x_train.shape

(6392, 64)

In [9]:
X_train=x_train
X_test=x_test

X_train=np.array(X_train)
X_test=np.array(X_test)

scaler = StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [10]:
X_train = X_train.reshape(-1, 64)
X_test = X_test.reshape(-1, 64)

In [11]:
X_train[0].shape

(64,)

In [12]:
X_train[0]

array([-0.67227689, -0.83764206,  0.95605425,  1.08311398,  0.6795567 ,
       -0.60189537, -1.41054795,  0.32010942,  1.02030015, -0.39404589,
       -0.66695237, -0.98740442, -0.65866832,  0.57291383, -0.36202094,
       -0.39437138, -0.56879442, -0.66761927, -0.34890317,  0.21840808,
        2.32312616, -0.33691954, -0.26944481, -0.35125928,  0.43038577,
        0.49466593,  1.168141  , -0.89121979,  0.19910309, -0.58878021,
       -0.68118186, -0.37228695,  0.42970793,  2.4229674 ,  1.56600146,
        1.74175605,  1.26851388, -1.26602971, -0.90468723, -1.67509351,
       -0.73478412, -0.07224572, -1.28535977, -0.27253855, -0.2642735 ,
        0.72212085, -0.4457816 , -0.54501542,  0.12878669, -0.19802357,
        0.00282557,  0.21495179, -0.25292066, -0.57601368, -0.78970703,
       -0.94668461,  0.36899895,  0.95022572, -0.12631669, -0.13554233,
       -0.9248761 , -1.13408584,  2.51292299,  0.30132673])

In [19]:
epochs=5000
batch_size=1024
learning_rate = 0.01
# model
model = Sequential()

input_shape = (64,1)#(128, 87, 1)
# model.add(Dense(128, activation='relu'))
# model.add(Dense(6, activation='softmax'))
model.add(Dense(256,input_dim=64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.build(input_shape)
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=learning_rate),
              metrics=['accuracy'])
# fit the model
history=model.fit(X_train, y_train_hot,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_hot))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               16640     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 771       
Total params: 17,411
Trainable params: 17,411
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5000


/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



7/7 [==============================] - 0s 18ms/step - loss: 1.2227 - accuracy: 0.3373 - val_loss: 1.3399 - val_accuracy: 0.2586
Epoch 2/5000
7/7 [==============================] - 0s 4ms/step - loss: 1.2086 - accuracy: 0.3526 - val_loss: 1.3242 - val_accuracy: 0.2676
Epoch 3/5000
7/7 [==============================] - 0s 4ms/step - loss: 1.1949 - accuracy: 0.3620 - val_loss: 1.3089 - val_accuracy: 0.2767
Epoch 4/5000
7/7 [==============================] - 0s 4ms/step - loss: 1.1816 - accuracy: 0.3736 - val_loss: 1.2936 - val_accuracy: 0.2847
Epoch 5/5000
7/7 [==============================] - 0s 4ms/step - loss: 1.1685 - accuracy: 0.3833 - val_loss: 1.2791 - val_accuracy: 0.2948
Epoch 6/5000
7/7 [==============================] - 0s 4ms/step - loss: 1.1559 - accuracy: 0.3925 - val_loss: 1.2648 - val_accuracy: 0.2998
Epoch 7/5000
7/7 [==============================] - 0s 4ms/step - loss: 1.1436 - accuracy: 0.4011 - val_loss: 1.2513 - val_accuracy: 0.3109
Epoch 8/5000
7/7 [==============

In [20]:
model.save('/Users/vedant/Desktop/Programming/ScreamDetection/CNN/trained_models/fcnn_batch-1024_epochs_5000_lr_0_01')

INFO:tensorflow:Assets written to: /Users/vedant/Desktop/Programming/ScreamDetection/CNN/trained_models/fcnn_batch-1024_epochs_5000_lr_0_01/assets


In [14]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [15]:
training_loss=history.history['loss']
validation_loss=history.history['val_loss']
training_acc=history.history['accuracy']
validation_acc=history.history['val_accuracy']

import pandas as pd
df=pd.DataFrame()
df['training_loss'] = training_loss
df['validation_loss'] = validation_loss
df['training_acc'] = training_acc
df['validation_acc'] = validation_acc

In [16]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
trace1 = go.Scatter(
    x=df.index,
    y=df['training_loss'],
    name='Training Loss'
)
trace2 = go.Scatter(
    x=df.index,
    y=df['validation_loss'],
    name='Validation Loss'
)
trace3 = go.Scatter(
    x=df.index,
    y=df['training_acc'],
    name='Training Accuracy',
    yaxis='y2'
)
trace4 = go.Scatter(
    x=df.index,
    y=df['validation_acc'],
    name='Validation Accuracy',
    yaxis='y2'
)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3,secondary_y=True)
fig.add_trace(trace4,secondary_y=True)
fig